In [2]:
import os
import pandas as pd
from py2neo import Graph, Node, Relationship

In [4]:
# Connect to the Neo4j database
NEO4J_URL = os.getenv('NEO4J_URL')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
graph = Graph(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [2]:
from py2neo import Graph, Node, Relationship
import pandas as pd
 
# Load the main data
file_path = "/data.xlsx"
data = pd.read_excel(file_path, header=0)
data1 = pd.read_excel(file_path, sheet_name="Inventory_data", header=0)
 
# Dictionary to track already created LCI Inventory Location nodes
created_inventory_locations = {}

# Iterate through the main dataset
for index, row in data.iterrows():
    # Create nodes with attributes
    Database = Node("Database", name=str(row['Database']))
    Chemical = Node("Target_Chemical", name=str(row['Target chemical']), formula=str(row['Formula']), molecular_weight=str(row['Molecular weight']), density=str(row['Density']), CAS_number=row['CAS number'])
    Title = Node("Paper_Title", name=str(row['Title']), author=str(row['Author']), published_year=row['Published year'], doi=str(row['Doi']))
    System_Boundary = Node("System_Boundary", name=str(row['System boundary']))
    FunctionalUnit_Product = Node("FunctionalUnit_Product", name=str(row['FunctionalUnit Product']))
    FunctionalUnit_Quantity = Node("FunctionalUnit_Quantity", name=str(row['FunctionalUnit Quantity']))
    FunctionalUnit_Unit = Node("FunctionalUnit_Unit", name=str(row['FunctionalUnit Unit']))
    Impact_Assessment_Method = Node("Impact_Assessment_Method", name=str(row['Impact assessment method']), impact_category=str(row['Impact category']))
    Pathway = Node("Pathway", name=str(row['LCI inventory pathway']), Inventory_ID=row['ID'], Target_product=str(row['Target chemical']))
 
    # Check if the LCI Inventory Location node already exists within the same title context
    inventory_location_name = str(row['LCI inventory location'])
    inventory_key = row['Title']+inventory_location_name
 
    if inventory_key not in created_inventory_locations.keys():
        # If not, create and store it in the dictionary
        created_inventory_locations[inventory_key] = Node("LCI_Table", name=inventory_location_name, page=str(row['Page_number']))
    else:
        # If it exists within the same title context, retrieve it from the dictionary
        LCI_Inventory_Location = created_inventory_locations[inventory_key]
 
    # Build relationship objectives
    Has_Chemical_Names = Relationship.type('Has_Chemical_Names')
    Has_LCA_Studies = Relationship.type('Has_LCA_Studies')
    Has_System_Boundary = Relationship.type("Has_System_Boundary")
    Has_Functional_Unit = Relationship.type("Has_Functional_Unit")
    Has_Reference_Product = Relationship.type("Has_Reference_Product")
    Has_Reference_Unit = Relationship.type("Has_Reference_Unit")   
    Has_LCI_Pathway = Relationship.type("Has_Pathway")
    Has_LCIA_Method = Relationship.type("Has_LCIA_Method")
 
    # Build relationships
    r1 = Has_Chemical_Names(Database, Chemical, name="Has_Chemical_Names")
    r2 = Has_LCA_Studies(Chemical, Title, name="Has_LCA_Studies")
    r3 = Has_System_Boundary(Title, System_Boundary, name='Has_System_Boundary')
    r4 = Has_Functional_Unit(Title, FunctionalUnit_Quantity, name='Has_Functional_Unit')
    r5 = Has_Reference_Unit(FunctionalUnit_Quantity, FunctionalUnit_Unit, name='Has_Reference_Unit')
    r6 = Has_Reference_Product(FunctionalUnit_Unit, FunctionalUnit_Product, name='Has_Reference_Product')
    r7 = Relationship(Title, "Has_Inventory_Table", created_inventory_locations[inventory_key])
    r8 = Has_LCI_Pathway(created_inventory_locations[inventory_key], Pathway, name='Has_Pathway')
    r9 = Has_LCIA_Method(Title, Impact_Assessment_Method, name='Has_LCIA_Method')
 
    # Merge nodes based on attributes
    graph.merge(Database, "Database", "name") 
    graph.merge(Chemical, "Target_Chemical", "name")  
    graph.merge(Title, "Paper_Title", "name") 
    graph.merge(System_Boundary, "System_Boundary", "name") 
    graph.merge(Pathway, "Pathway", "Inventory_ID")
 
    # Create relationships
    graph.create(r1 | r2 | r3 | r4 | r5 | r6 | r7 | r8 | r9) 
 
# Run cleanup queries
query1 = """
match (p:`Paper_Title`) - [:Has_LCIA_Method]->(i:`Impact_Assessment_Method`)
WITH p, collect(i) AS methods
WHERE size(methods) > 1
WITH p, methods, head(methods) AS keepNode, tail(methods) AS deleteNodes
SET keepNode.someProperty = keepNode.someProperty + ' ' + deleteNodes[0].someProperty
FOREACH (n IN deleteNodes | DETACH DELETE n)
RETURN p.title AS PaperTitle, keepNode
"""
 
query2 = """
match (p:`Paper_Title`) - [:Has_Functional_Unit]->(f:`FunctionalUnit_Quantity`)
WITH p, collect(f) AS methods
WHERE size(methods) > 1
WITH p, methods, head(methods) AS keepNode, tail(methods) AS deleteNodes
SET keepNode.someProperty = keepNode.someProperty + ' ' + deleteNodes[0].someProperty
FOREACH (n IN deleteNodes | DETACH DELETE n)
RETURN p.title AS PaperTitle, keepNode
"""
 
query3 = """
MATCH (u:`FunctionalUnit_Unit`)
WHERE NOT (:`FunctionalUnit_Quantity`)-[:Has_Reference_Unit]->(u)
OPTIONAL MATCH (u)-[:Has_Reference_Product]->(p:`FunctionalUnit Product`)
DETACH DELETE u, p
RETURN COUNT(u) AS UnitsDeleted, COUNT(p) AS ProductsDeleted
"""
 
query4 = """
MATCH (n)
WHERE n.name = '/'
DETACH DELETE n
RETURN COUNT(n) AS DeletedNodes
"""
graph.run(query1)
graph.run(query2)
graph.run(query3)
graph.run(query4)
 
# Iterate through the inventory dataset
for index, row in data1.iterrows():
    # Create nodes with attributes
    Pathway = Node("Pathway", name=str(row['Pathway']), Inventory_ID=row['ID'])
    Activity = Node("Activity", name=str(row['Activity']), Inventory_ID=row['ID'])
    Flow = Node("Flow", name=str(row['Flow name']), is_reference_flow=row['Is_reference_flow'])
    Flow_Amount = Node("Amount", name=row['Amount'])
    Flow_Unit = Node("Unit", name=str(row['Unit']))
 
    # Build relationship objectives
    Has_Activity = Relationship.type('Has_Activity')
    Has_Input_Flow = Relationship.type("Has_Input_Flow")
    Has_Output_Flow = Relationship.type("Has_Output_Flow")
    Has_Value = Relationship.type("Has_Value")
    Has_Unit = Relationship.type("Has_Unit")
 
    # Build relationship and add attributes
    r1 = Has_Activity(Pathway, Activity, name="Has_Activity")
 
    if row['Type'] == 'Has_Input':
        r2 = Has_Input_Flow(Activity, Flow, name="Has_Input_Flow")
    elif row['Type'] == 'Has_Output':
        r3 = Has_Output_Flow(Activity, Flow, name="Has_Output_Flow")
 
    r4 = Has_Value(Flow, Flow_Amount, name="Has_Value")
    r5 = Has_Unit(Flow_Amount, Flow_Unit, name="Has_Unit")
 
    # Merge nodes based on attributes
    graph.merge(Pathway, "Pathway", "Inventory_ID")  
    graph.merge(Activity, "Activity", "Inventory_ID")
 
    graph.create(r1 | r2 | r3 | r4 | r5)
 
print("Finished!")

Finished!
